<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema3_Tarea3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tema 3 - Tarea 3

In [ ]:
# Importamos funciones y establecemos configuración general
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = [10,4]  # Default figure size

In [ ]:
# Definimos funciones a usar en este código:
def forecast_accuracy(forecast, actual):
    rmse = np.sqrt(np.mean((forecast-actual)**2))
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    print({'rmse':rmse,'mape':mape,'corr':corr})

def DFtest(datos):
    print('Results of Dickey-Fuller Test (H0= NO estac.):')
    dftest = adfuller(datos, autolag='AIC')
    print('Test Statistic', dftest[0])
    print('p-value', dftest[1])
    print('#Lags Used', dftest[2])
    print('# obs', dftest[3])      
    for k, v in dftest[4].items():
        print(k,v)

In [ ]:
# Importamos datos
f="https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/Tema3_Tarea3.xlsx"
pib = pd.read_excel(f,"pib",index_col=0, parse_dates=True) # Preparar columna excel XXXXQX

In [ ]:
#Training y test dataset
train=pib[:85]
test=pib[85:]

### Modelo SARIMA

In [ ]:
# GRÁFICO
fig, axes = plt.subplots(2, 1, figsize=(10,5), dpi=100, sharex=True)
# Diferenciación normal
axes[0].plot(train, label='Serie original') # con to_timestamp()
axes[0].plot(train.diff(1), label='Diferenciación normal')
axes[0].set_title('Diferenciación normal')
axes[0].legend(loc='upper left', fontsize=10)
# Diferenciación estacional
axes[1].plot(train, label='Serie original')
axes[1].plot(train.diff(4), label='Diferenciación estacional', color='green')
axes[1].set_title('Diferenciación estacional')
plt.legend(loc='upper left', fontsize=10)
plt.suptitle('PIB TRIMESTRAL', fontsize=16)
plt.show()

In [ ]:
# Dibujamos correlogramas y usamos el test DF
fig, ax = plt.subplots(2, figsize=(12,6))
ax[0] = plot_acf(train.diff(4).dropna(), ax=ax[0], lags=28)
ax[1] = plot_pacf(train.diff(4).dropna(), ax=ax[1], lags=28)
plt.show()
DFtest(train.pibv.diff(4).dropna())

* La serie es no estacionaria por lo que hay que tomar primeras diferencias. El autocorrelograma muestra claramente que la autocorrelación de los retardos cae lentamente y el p-value de test de raíz unitaria es 0.10.

In [ ]:
# Representamos correlogramas de serie diferenciada de nuevo:
fig, ax = plt.subplots(2, figsize=(12,6))
ax[0] = plot_acf(train.diff(4).diff().dropna(), ax=ax[0], lags=28)
ax[1] = plot_pacf(train.diff(4).diff().dropna(), ax=ax[1], lags=28)
plt.show()
DFtest(train.pibv.diff(4).diff().dropna())

* Ahora sí es estacionaria.
* Modelo SARIMA = (1,1,1)(0,1,0,4).

In [ ]:
# SARIMA(1,1,1)(0,1,0,4)
mod = sm.tsa.statespace.SARIMAX(train,order=(1,1,1),seasonal_order=(0,1,0,4), trend="c",
                                enforce_stationarity=False, enforce_invertibility=False)
results= mod.fit()
print(results.summary())

In [ ]:
# Errores diagnóstico
results.plot_diagnostics();plt.show()

In [ ]:
# Cómo de preciso es el modelo para predecir??
pred = results.get_prediction('2016-04-01','2019-04-01', dynamic=False)
forecast_accuracy(pred.predicted_mean, test.pibv) # La función autoarima (pmdarima) no parece mejorar ni el AIC ni la exactitud de la predicción (resultados no mostrados)

In [ ]:
# Dibujamos la predicción:
pred_ci = pred.conf_int()
index_of_fc = pd.date_range(start='2016-04-01',end='2019-04-01', freq='QS-OCT')
# Creando las series para su representación gráfica
fitted_series = pd.Series(pred.predicted_mean, index=index_of_fc)
lower_series = pd.Series(pred_ci["lower pibv"], index=index_of_fc)
upper_series = pd.Series(pred_ci["upper pibv"], index=index_of_fc)
# Gráfico - 
plt.figure(figsize=(12,8))
plt.plot(train["2011-01-01":], label="Training period", color="b") 
plt.plot(test, label="Actual values", color="k")
plt.plot(fitted_series, label="Predicted", color='r')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)
plt.legend()
plt.title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

In [ ]:
# Y también predecimos hacia el futuro 4 periodos
results = sm.tsa.statespace.SARIMAX(pib,order=(1,1,1),seasonal_order=(0,1,0,4), trend="c",
                                enforce_stationarity=False, enforce_invertibility=False).fit()
pred = results.get_prediction('2019-07-01','2020-04-01', dynamic=False)

In [ ]:
# Dibujamos la predicción:
pred_ci = pred.conf_int()
index_of_fc = pd.date_range(start='2019-07-01',end='2020-04-01', freq='QS-OCT')
# Creando las series para su representación gráfica
fitted_series1 = pd.Series(pred.predicted_mean, index=index_of_fc)
lower_series = pd.Series(pred_ci["lower pibv"], index=index_of_fc)
upper_series = pd.Series(pred_ci["upper pibv"], index=index_of_fc)
# Gráfico - 
plt.figure(figsize=(12,8))
plt.plot(pib["2011-01-01":],color="b") 
plt.plot(fitted_series1, label="Predicted", color='r')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)
plt.legend()
plt.title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

### Modelo SARIMAX

In [ ]:
# Creamos la tabla con los indicadores
otros = pd.read_excel(f,"otros",index_col=5, parse_dates=True) # Preparar columna excel XXXX-XX 
otros.index=otros.index.to_period("M") # Indicar que son datos mensuales explícitamente (hace falta luego para el resample, o sea, pasarlos a trimestral)
otros.drop("periodo", axis=1, inplace=True)

In [ ]:
#Resample (pasar los datos de mensuales a trimestrales)
otrosq=otros.resample("Q", label="right").sum() # se agrega por trimestres sumando los meses
otrosq.indvent=otrosq.indvent/3 # Para este indicator tiene sentido tomar la media, por eso se divide entre tres.
otrosq.isa=otrosq.isa/3 # Para este indicator tiene sentido tomar la media, por eso se divide entre tres.
otrosq.index=otrosq.index.to_timestamp() # Para que esté en igual formato temporal que la dataframe pib

In [ ]:
# Correlaciones- para ver qué variable usar como exógena
# Las correlaciones se calculan usando las tasas de variación (como se indica en el enunciado)
vpib=pib.pibv/pib.pibv.shift(4)-1
for c in otrosq.columns:
    #print(c,": corr in levels: ",np.corrcoef(pib.pibv, otrosq[c][:98])[0,1]) 
    otrosq["v"+c]=otrosq[c]/otrosq[c].shift(4)-1
    print(c,":corr var interanual: ",np.corrcoef(vpib.dropna(), otrosq["v"+c][:-1].dropna())[0,1])     
plt.scatter(vpib,otrosq.visa[:-1])
# Vemos que el indicador más correlacionado es ISA
isa=otrosq["isa"].copy()

In [ ]:
# Vemos que el indicador más correlacionado es ISA
isa=otrosq["isa"].copy()

In [ ]:
# CONSTRUIMOS MODELO SARIMAX
exog_in=isa[:85]
mod = sm.tsa.statespace.SARIMAX(train,exog=exog_in, order=(1,1,1),seasonal_order=(0,1,0,4), trend="c",
                                enforce_stationarity=False, enforce_invertibility=False)
results= mod.fit()
print(results.summary())

In [ ]:
# Errores diagnóstico
results.plot_diagnostics();plt.show()

In [ ]:
# Cómo de preciso es el modelo para predecir??
exog_out=isa['2016-04-01':'2019-04-01'].to_frame() # convierto a dataframe porque si no no funciona el comando siguiente
pred = results.get_prediction('2016-04-01','2019-04-01', exog=exog_out, dynamic=False)
forecast_accuracy(pred.predicted_mean, test.pibv)

In [ ]:
# Dibujamos la predicción:
pred_ci = pred.conf_int()
index_of_fc = pd.date_range(start='2016-04-01',end='2019-04-01', freq='QS-OCT')
# Creando las series para su representación gráfica
fitted_series = pd.Series(pred.predicted_mean, index=index_of_fc)
lower_series = pd.Series(pred_ci["lower pibv"], index=index_of_fc)
upper_series = pd.Series(pred_ci["upper pibv"], index=index_of_fc)
# Gráfico - 
plt.figure(figsize=(12,8))
plt.plot(train["2011-01-01":], label="Training period", color="b") 
plt.plot(test, label="Actual values", color="k")
plt.plot(fitted_series, label="Predicted", color='r')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)
plt.legend()
plt.title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

In [ ]:
# Y también predecimos hacia el futuro 1 periodo
results = sm.tsa.statespace.SARIMAX(pib,exog=isa[:-1], order=(1,1,1),seasonal_order=(0,1,0,4), trend="c",
                                enforce_stationarity=False, enforce_invertibility=False).fit()
pred = results.get_prediction('2019-07-01','2019-07-01', exog=isa[-1:].to_frame(), dynamic=False)

In [ ]:
# Dibujamos la predicción:
pred_ci = pred.conf_int()
index_of_fc = pd.date_range(start='2019-07-01',end='2019-07-01', freq='QS-OCT')
# Creando las series para su representación gráfica
fitted_series2 = pd.Series(pred.predicted_mean, index=index_of_fc)
lower_series = pd.Series(pred_ci["lower pibv"], index=index_of_fc)
upper_series = pd.Series(pred_ci["upper pibv"], index=index_of_fc)
# Gráfico - 
plt.figure(figsize=(12,8))
plt.plot(pib["2011-01-01":],color="b") 
plt.plot(index_of_fc[-1:],fitted_series2.iloc[0],marker='o') # Punto estimado
plt.plot(index_of_fc[-1:],lower_series.iloc[0], marker='.') # IC inferior
plt.plot(index_of_fc[-1:],upper_series.iloc[0], marker='.') # IC superior
plt.title("SARIMA - Predicción de ventas de medicamentos")
plt.show()

### ¿Qué estimación del PIB es más acertada?

In [ ]:
#Valores de PIB:
sarima3q19=fitted_series1[-4:-3][0]
sarimax3q19=fitted_series2[-1:][0]
pib_real3q19=110.1652
pib_real3q18=pib.pibv[-4:-3][0]

In [ ]:
print("Estim. SARIMA=",sarima3q19)
print("Estim. SARIMAX=",sarimax3q19)
print("Pib real 2019q3=",pib_real3q19)
print("Var interanual SARIMA=",np.round(sarima3q19/pib_real3q18-1, decimals=4))
print("Var interanual SARIMAX=",np.round(sarimax3q19/pib_real3q18-1, decimals=4))
print("Var interanual real=",np.round(pib_real3q19/pib_real3q18-1, decimals=4))